### Pandas中的DataFrame数据结构
#### DataFrame数据结构的概念及特征
DataFrame是一种表格型数据结构，它含有一组有序的列，每列可以是不同的值类型（数值、字符串、布尔值等）
DataFrame既有行索引又有列索引，可以看作是由Series组成的字典。
DataFrame的数据是以一个或多个**二维块**存放的（而不是列表，字典或别的一维数据结构）
#### 构建DataFrame的方法
1. 常用的方法传入一个由等长列表或Numpy数组组成的字典

In [1]:
data = {'xing':['zhao','hu','ma'], 'ming':['jing','long','wei'],'zhi':['teng','fei','qin']}

In [3]:
import pandas as pn
frame = pn.DataFrame(data)

In [4]:
frame

,xing,ming,zhi
0,zhao,jing,teng
1,hu,long,fei
2,ma,wei,qin


结果DataFrame会自动加上索引（跟Series一样），且全部列会被有序排列
2. 另一种构造DataFrame的方式：使用嵌套字典（字典的字典）

In [57]:
data2 = {'xing':{2006:'zhao',2007:'hu'},'ming':{2007:'jing',2009:'long'}}

In [58]:
frame3 = pn.DataFrame(data2)

In [59]:
frame3

,xing,ming
2006,zhao,NaN
2007,hu,jing
2009,NaN,long


可以看出外层字典的键作为列，内层键作为行索引；内层字典的键会被合并、排序以形成最终的索引，如果显示指定了索引，则不会这样

In [60]:
pn.DataFrame(data2, index=pn.Series([2006,2007,2009,2008]))

,xing,ming
2006,zhao,NaN
2007,hu,jing
2009,NaN,long
2008,NaN,NaN


3.第三种生成DataFrame的方法:使用Series生成
由Series组成的字典差不多也是一样的用法

In [63]:
pdata = {'xing':frame3['xing'],'ming':frame3['ming']}

In [64]:
pn.DataFrame(pdata)

,xing,ming
2006,zhao,NaN
2007,hu,jing
2009,NaN,long


下面列出DataFrame构造函数所能接受的各种数据
![](/images/7178691-106835b28c0cea5a.png)

#### DataFrame中的columns属性类似于Series的index
如果指定了列序列（columns属性），则DataFrame的列就会按照指定顺序进行排列

In [7]:
frame1 = pn.DataFrame(data, columns=['zhi','xing','ming'])

In [8]:
frame1

,zhi,xing,ming
0,teng,zhao,jing
1,fei,hu,long
2,qin,ma,wei


跟Series一样，如果传入的列在数据中找不到，就会产生NA值

In [9]:
frame2 = pn.DataFrame(data, columns=['zhi','ming','miss'], index=['one','two','three'])

In [10]:
frame2

,zhi,ming,miss
one,teng,jing,NaN
two,fei,long,NaN
three,qin,wei,NaN


In [11]:
frame2.columns

Index(['zhi', 'ming', 'miss'], dtype='object')

#### 如何获取DataFrame的列
通过类似**字典标记的方式**或**属性的方式**可以获取DataFrame的列

In [12]:
frame2['zhi']

one      teng
two       fei
three     qin
Name: zhi, dtype: object

In [14]:
frame2.zhi

one      teng
two       fei
three     qin
Name: zhi, dtype: object

>注：返回的Series拥有原有DataFrame相同的索引，切其**name**属性也已经被相应地设置好

#### 如何获取DataFrame的行
通过**位置**或**名称**的方式获取，比如用索引字段**loc**

In [20]:
frame2.loc['three']

zhi     qin
ming    wei
miss    NaN
Name: three, dtype: object

#### 列的赋值
列可以通过赋值的方式进行修改。
1. 直接赋值一个标量或一组值

In [22]:
frame2['miss'] = 16.5

In [23]:
frame2

,zhi,ming,miss
one,teng,jing,16.5
two,fei,long,16.5
three,qin,wei,16.5


In [27]:
import numpy as np
frame2['miss'] = np.arange(3)

In [28]:
frame2

,zhi,ming,miss
one,teng,jing,0
two,fei,long,1
three,qin,wei,2


> 注：将列表或数组赋值给某一列时，其长度必须跟DataFrame的长度相匹配

2. 赋值Series

In [30]:
val = pn.Series([1.4,2.5,3.5], index = ['one','two','four'])

In [33]:
frame2['miss'] = val
frame2['new'] = val

In [34]:
frame2

,zhi,ming,miss,new
one,teng,jing,1.4,1.4
two,fei,long,2.5,2.5
three,qin,wei,NaN,NaN


> 注：赋值是Series时，就会精确匹配，DataFrame的索引所有的空位都会被填上缺失值,同时为不存在的列赋值会创建一个新列

#### 删除列（del关健字）

In [36]:
frame2['shanchu'] = frame2.ming == 'zhao'

In [37]:
frame2

,zhi,ming,miss,new,shanchu
one,teng,jing,1.4,1.4,False
two,fei,long,2.5,2.5,False
three,qin,wei,NaN,NaN,False


In [38]:
del frame2['shanchu']

In [39]:
frame2.columns

Index(['zhi', 'ming', 'miss', 'new'], dtype='object')

> 注：通过索引方式返回的列只是相应数据的视图而已，并不是副本，所以对返回的Series所做的任何修改都会反映到DataFrame上，通过Series的copy方法可显式的复制制

#### DataFrame进行转置

In [45]:
frame3.T

,2006,2007,2008,2009
xing,zhao,hu,NaN,NaN
ming,NaN,NaN,jing,long


#### DataFrame的name属性
如果设置了DataFrame的index和columns的name属性，则这些信息也会显示出来：

In [65]:
frame3.index.name = 'year'

In [66]:
frame3.columns.name = 'state'

In [67]:
frame3

state,xing,ming
year,,
2006,zhao,NaN
2007,hu,jing
2009,NaN,long


#### DataFrame的values属性
跟Series一样，values属性也会以二维ndarray的形式返回DataFrame的数据

In [68]:
frame3.values

array([['zhao', nan],
       ['hu', 'jing'],
       [nan, 'long']], dtype=object)

这里DataFrame各列的数据类型不同，所以其数据类型是选用兼容所有列的数据类型，是一种嵌套dtype来表示